## Imports

In [1]:
import time

import numpy as np
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt
from numpy.random import uniform
import pdb
from tqdm import tqdm



## Show functions

In [24]:
def show_image_cv(title,image,fx=0.2,fy=0.2,output = True):

    if not output:
        return

    image = cv.resize(image,(0,0),fx=fx,fy=fy)
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def show_image_matplot(title, image,fx=0.3,fy=0.3,output = True):
    if not output:
        return

    image_resized = cv.resize(image, (0, 0), fx=fx, fy=fy)

    image = cv.cvtColor(image_resized, cv.COLOR_BGR2RGB)

    plt.imshow(image)
    plt.title(title)
    plt.axis('off')  # Ascunde axele pentru a afișa doar imaginea
    plt.show()

## Directory and files functions

In [3]:
def get_image_paths(dir,index):
    image_paths = os.listdir(dir)
    image_paths = ["./" + dir + "/" + file for file in image_paths if file[0:2] == str(index) + "_" and file[-4:] == ".jpg"]
    return sorted(image_paths)


## HSV masking

In [4]:
# hsv mask

def get_outer_masked_image(img,output = False):
    lower_hsv_bound = np.array([20,0,0])
    upper_hsv_bound = np.array([255,255,255])
    hsv_image = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    mask = cv.inRange(hsv_image, lower_hsv_bound, upper_hsv_bound)
    show_image_cv("mask",mask,output = output)

    new_image = cv.bitwise_and(img, img, mask=mask)
    show_image_cv("ceva",new_image,output = output)

    return new_image



## Get game contour function


In [5]:
def get_game_countour(image,output = False):

    masked_image = get_outer_masked_image(image)
    masked_image_grey = cv.cvtColor(masked_image, cv.COLOR_BGR2GRAY)

    show_image_cv("masked_image_grey",masked_image_grey,output = output)

    masked_image_grey = cv.medianBlur(masked_image_grey,3)
    show_image_cv('median_blur_image',masked_image_grey,output = output)

    contours, _ = cv.findContours(masked_image_grey,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    max_area = 0
    for i in range(len(contours)):
            if(len(contours[i]) >3):
                possible_top_left = None
                possible_bottom_right = None
                for point in contours[i].squeeze():
                    if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                        possible_top_left = point

                    if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                        possible_bottom_right = point

                diff = np.diff(contours[i].squeeze(), axis = 1)
                possible_top_right = contours[i].squeeze()[np.argmin(diff)]
                possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
                if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                    max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                    top_left = possible_top_left
                    bottom_right = possible_bottom_right
                    top_right = possible_top_right
                    bottom_left = possible_bottom_left


    image_copy = cv.cvtColor(masked_image_grey.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    show_image_cv("detected corners",image_copy,output = output)


    width = 810
    height = 810

    puzzle = np.array([top_left,top_right,bottom_left,bottom_right],dtype=np.float32)
    destination = np.array([[0,0],[width,0],[0,height],[width,height]],dtype=np.float32)
    M = cv.getPerspectiveTransform(puzzle,destination)
    result = cv.warpPerspective(image,M,(width,height))

    return result

## Game table extractor

In [6]:
def extract_game(image_paths,size = 1000):
    images = []
    size = size if size <= len(image_paths) else len(image_paths)
    for i in tqdm(range(size)):
        img = cv.imread(image_paths[i])
        result = get_game_countour(img)
        images.append(result)
    return np.array(images)

def strip_margins(img,output = False):

    img = img.copy()
    img_grey = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    BLACK = 0

    height,width,_ = img.shape

    # somewhere between 13% and 14% of the height and width of the image
    top_margin = 107
    bottom_margin = height - 102
    left_margin = 107
    right_margin = width - 105

    img_grey[:top_margin, :] = BLACK
    img_grey[bottom_margin:, :] = BLACK

    img_grey[:, :left_margin] = BLACK
    img_grey[:, right_margin:] = BLACK

    show_image_cv("ceva",img_grey,fx=1,fy=1,output = output)

    masked_image_grey = img_grey

    contours, _ = cv.findContours(masked_image_grey,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    max_area = 0
    for i in range(len(contours)):
            if(len(contours[i]) >3):
                possible_top_left = None
                possible_bottom_right = None
                for point in contours[i].squeeze():
                    if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                        possible_top_left = point

                    if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                        possible_bottom_right = point

                diff = np.diff(contours[i].squeeze(), axis = 1)
                possible_top_right = contours[i].squeeze()[np.argmin(diff)]
                possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
                if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                    max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                    top_left = possible_top_left
                    bottom_right = possible_bottom_right
                    top_right = possible_top_right
                    bottom_left = possible_bottom_left


    image_copy = cv.cvtColor(masked_image_grey.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),2,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),2,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),2,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),2,(0,0,255),-1)
    show_image_cv("detected corners",image_copy,output = output,fx=1,fy=1)


    width = 14 * 60
    height = 14 * 60

    puzzle = np.array([top_left,top_right,bottom_left,bottom_right],dtype=np.float32)
    destination = np.array([[0,0],[width,0],[0,height],[width,height]],dtype=np.float32)
    M = cv.getPerspectiveTransform(puzzle,destination)
    result = cv.warpPerspective(img,M,(width,height))

    return result

def extract_relevant_game(images):
    new_images = []

    for image in tqdm(images):
        new_images.append(strip_margins(image))

    return np.array(new_images)


In [7]:
game_tables = extract_game(get_image_paths("antrenare",1))

100%|██████████| 50/50 [00:14<00:00,  3.41it/s]


In [8]:
relevant_game_tables = extract_relevant_game(game_tables)

100%|██████████| 50/50 [00:00<00:00, 318.86it/s]


## Cell delimiter function

In [9]:
def draw_lines(images):

    new_images = images.copy()

    lines_horizontal = []
    for i in range(0,14*60+1,60):
        line = []
        line.append((0,i))
        line.append((14*60-1,i))
        lines_horizontal.append(line)

    lines_vertical = []
    for i in range(0,14*60+1,60):
        line = []
        line.append((i,0))
        line.append((i,14*60-1))
        lines_vertical.append(line)

    GREEN = (0,255,0)
    RED = (0,0,255)

    for image in tqdm(new_images):
        for line in lines_vertical:
            cv.line(image,line[0],line[1],GREEN,2)

        for line in lines_horizontal:
            cv.line(image,line[0],line[1],RED,2)

    return new_images


In [10]:
delimited_tables = draw_lines(relevant_game_tables)

for image in delimited_tables:
    show_image_cv("cevas",image,fx=1,fy=1)
    break

100%|██████████| 50/50 [00:00<00:00, 2413.68it/s]


In [13]:
paths = ["./imagini_auxiliare/" + file for file in os.listdir("./imagini_auxiliare")]
test_tables = extract_game(paths)
game_tables = extract_relevant_game(test_tables)

lines = draw_lines(game_tables)

100%|██████████| 4/4 [00:00<00:00, 1975.42it/s]


In [31]:
show_image_cv("",lines[2],fx=1,fy=1)

In [46]:

def store_template_numbers():

    img = ["./imagini_auxiliare/03.jpg"]
    table = extract_game(img)
    game_table = extract_relevant_game(table)[0]

    piece_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 50, 54, 56, 60, 63, 64, 70, 72, 80, 81, 90]

    index = 0

    for i in range(5 * 60, 9 * 60 +1 ,60):
        for j in range(4 * 60 , 12 * 60, 60):
            frame = game_table[i:i+60,j:j+60,:]
            # show_image_matplot("",frame)
            cv.imwrite("./templates/" + str(piece_values[index])+ ".jpg", frame)
            index += 1

    i = 10 * 60

    for j in range(4 * 60 , 10 * 60, 60):
            frame = game_table[i:i+60,j:j+60,:]
            # show_image_matplot("",frame)
            cv.imwrite("./templates/" + str(piece_values[index])+ ".jpg", frame)
            index +=1

store_template_numbers()

100%|██████████| 1/1 [00:00<00:00, 526.92it/s]
